# Import

In [8]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

In [9]:
load_dotenv() 
DATASET_FOLDER = os.getenv('DATASET_FOLDER')

In [10]:
raw_df = pd.read_csv(f"{DATASET_FOLDER}/Food_Establishment_Inspection_Data.csv")

/tmp/ipykernel_42782/3624628935.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(f"{DATASET_FOLDER}/Food_Establishment_Inspection_Data.csv")


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255582 entries, 0 to 255581
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Name                        255582 non-null  object 
 1   Program Identifier          255582 non-null  object 
 2   Inspection Date             254921 non-null  object 
 3   Description                 255582 non-null  object 
 4   Address                     255582 non-null  object 
 5   City                        255582 non-null  object 
 6   Zip Code                    255582 non-null  object 
 7   Phone                       184808 non-null  object 
 8   Longitude                   255179 non-null  float64
 9   Latitude                    255179 non-null  float64
 10  Inspection Business Name    254921 non-null  object 
 11  Inspection Type             254921 non-null  object 
 12  Inspection Score            254882 non-null  float64
 13  Inspection Res

In [4]:
raw_df.describe()

,Longitude,Latitude,Inspection Score,Violation Points,Grade
count,255179.000000,255179.000000,254882.000000,255582.000000,193630.000000
mean,-122.197295,47.563608,14.094679,4.390485,1.418881
std,3.783447,0.128476,20.268395,6.105445,0.618843
min,-122.898939,47.043716,-2.000000,0.000000,1.000000
25%,-122.333585,47.479268,0.000000,0.000000,1.000000
50%,-122.299119,47.602468,5.000000,3.000000,1.000000
75%,-122.197717,47.661462,20.000000,5.000000,2.000000
max,122.384610,48.772284,180.000000,30.000000,4.000000


# Data Wrangling

In [12]:
raw_df.info() ,raw_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255582 entries, 0 to 255581
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Name                        255582 non-null  object 
 1   Program Identifier          255582 non-null  object 
 2   Inspection Date             254921 non-null  object 
 3   Description                 255582 non-null  object 
 4   Address                     255582 non-null  object 
 5   City                        255582 non-null  object 
 6   Zip Code                    255582 non-null  object 
 7   Phone                       184808 non-null  object 
 8   Longitude                   255179 non-null  float64
 9   Latitude                    255179 non-null  float64
 10  Inspection Business Name    254921 non-null  object 
 11  Inspection Type             254921 non-null  object 
 12  Inspection Score            254882 non-null  float64
 13  Inspection Res

(None,
            Longitude       Latitude  Inspection Score  Violation Points  \
 count  255179.000000  255179.000000     254882.000000     255582.000000   
 mean     -122.197295      47.563608         14.094679          4.390485   
 std         3.783447       0.128476         20.268395          6.105445   
 min      -122.898939      47.043716         -2.000000          0.000000   
 25%      -122.333585      47.479268          0.000000          0.000000   
 50%      -122.299119      47.602468          5.000000          3.000000   
 75%      -122.197717      47.661462         20.000000          5.000000   
 max       122.384610      48.772284        180.000000         30.000000   
 
                Grade  
 count  193630.000000  
 mean        1.418881  
 std         0.618843  
 min         1.000000  
 25%         1.000000  
 50%         1.000000  
 75%         2.000000  
 max         4.000000  )

In [15]:
cols = { k:k.replace(' ','_').lower() for k in raw_df.keys()}
cols
pre_df = raw_df.rename(columns=cols)

In [16]:
pre_df = pre_df.dropna()

In [17]:
pre_df.city=pre_df.city.str.upper()


In [25]:
del pre_df['program_identifier']

In [26]:
df = pre_df

# Pandas

In [27]:
df.head(2)

,name,inspection_date,description,address,city,zip_code,phone,longitude,latitude,inspection_business_name,...,inspection_score,inspection_result,inspection_closed_business,violation_type,violation_description,violation_points,business_id,inspection_serial_num,violation_record_id,grade
0,#807 TUTTA BELLA,03/02/2023,Seating 0-12 - Risk Category III,2746 NE 45TH ST,SEATTLE,98105,(206) 722-6400,-122.296415,47.662311,#807 TUTTA BELLA,...,20.0,Unsatisfactory,False,RED,1300 - Food contact surfaces cleaned and sanit...,15,PR0089260,DAJ5DTHLV,IVBTPZO0B,1.0
1,#807 TUTTA BELLA,03/02/2023,Seating 0-12 - Risk Category III,2746 NE 45TH ST,SEATTLE,98105,(206) 722-6400,-122.296415,47.662311,#807 TUTTA BELLA,...,20.0,Unsatisfactory,False,RED,2120 - Proper cold holding temperatures ( 42 d...,5,PR0089260,DAJ5DTHLV,IV5GOME67,1.0


In [28]:
uniques = {}
uniques_count={}
df.apply(
    (
        lambda x: (uniques.update(
            {x.name:x.unique()}
        ),
          uniques_count.update(
              {x.name:len(x.unique())} 
          ))
    ),
    axis=0)
uniques_count

{'name': 5349,
 'inspection_date': 4531,
 'description': 15,
 'address': 5840,
 'city': 41,
 'zip_code': 157,
 'phone': 4882,
 'longitude': 5601,
 'latitude': 5602,
 'inspection_business_name': 5349,
 'inspection_type': 3,
 'inspection_score': 134,
 'inspection_result': 10,
 'inspection_closed_business': 2,
 'violation_type': 2,
 'violation_description': 144,
 'violation_points': 7,
 'business_id': 6201,
 'inspection_serial_num': 42308,
 'violation_record_id': 94087,
 'grade': 4}

In [37]:
pd.pivot_table(df,index=['city','name','grade'],columns=['violation_type'],aggfunc='size')

violation_type                                       BLUE   RED
city        name                              grade            
ALGONA      COCO JOE'S                        1.0    56.0  37.0
            TACOS EL TAJIN                    1.0     NaN   1.0
AUBURN      7-ELEVEN # 38940                  3.0     NaN  10.0
            7-ELEVEN #27219C                  2.0     3.0   5.0
            7-ELEVEN #35333B                  2.0    11.0  16.0
...                                                   ...   ...
WOODINVILLE WOODIN CREEK KITCHEN & TAP        1.0     NaN   1.0
            WOODINVILLE BAGEL BAKERY          2.0     2.0   7.0
            WOODINVILLE CAFE                  1.0     6.0  17.0
            WOODINVILLE CUT SHOP              1.0     3.0   4.0
            Walla Walla Steak Co- Woodinville 2.0     NaN   1.0

[5815 rows x 2 columns]